#Learnings

Code source: https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

###import

In [0]:
# https://keras.io/
!pip install -q keras
import keras

### Connecting to google drive
Used for load .txt file from the drive

In [5]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 3.4MB/s 


### Small LSTM Network to Generate Text for Alice in Wonderland

Load necessary libs and load the text file from drive

In [6]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
import os

#https://mc.ai/how-to-save-and-upload-deep-learning-machine-learning-models-in-google-colab-using-google-drive/
downloaded = drive.CreateFile({'id':'1rReMgfp_-1gP7BgGN6923Gea5c0QgIzq'}) #https://drive.google.com/open?id=1rReMgfp_-1gP7BgGN6923Gea5c0QgIzq # NOTE: id is from sharable link
filename = "Wonderland1.txt" #/content/drive/My Drive/
downloaded.GetContentFile(filename)
raw_text = open(filename).read()

# load ascii text and covert to lowercase
raw_text = raw_text.lower()

W0726 21:25:00.376646 140388265539456 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import Lock

###Display few character of the raw text

In [8]:
first_50_letters = raw_text[:50]
print(first_50_letters)

﻿alice's adventures in wonderland

chapter i. down


###Remove punctuations from the raw text
###Display first 50 characters

In [9]:
import string
punctuations = set(string.punctuation)
raw_text = ''.join(ch for ch in raw_text if ch not in punctuations)
first_50_letters = raw_text[:50]
print(first_50_letters)

﻿alices adventures in wonderland

chapter i down t


###Convert char to int

In [10]:
#create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):

  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])

Total Characters:  136037
Total Vocab:  29


### pad_sequences with post padding is used 
https://machinelearningmastery.com/data-preparation-variable-length-input-sequences-sequence-prediction/

helps in forming equal length data, padding value is zero not a random value
(Train the model on padded sentences rather than random sequences of characters)

In [0]:
from keras.preprocessing.sequence import pad_sequences

dataX = pad_sequences(dataX, padding='post', maxlen=100)
n_patterns = len(dataX)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

### Constructing LSTM model
https://machinelearningmastery.com/how-to-reduce-overfitting-with-dropout-regularization-in-keras/
LSTM can be regularized by dropping different sequences(time steps) as follows:
  
    1. input sequence,   -> dropout used within LSTM layer
    2. recurrent input (h or feedback sequence),-> recurrent_dropout used within LSTM layer
    3. output sequence (which will be fed to proceeding layer) -> dropout used after LSTM layer


In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), dropout=0.1,return_sequences=True))# assignment add dropout of 0.1 to input layer
model.add(Dropout(0.1))# assignment dropout = 0.1
model.add(LSTM(256))
# model.add(Dropout(0.2)) # assignment remove dropout
model.add(Dense(y.shape[1], activation='softmax'))

###Compile and fit the model

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model.fit(X, y, epochs=10, batch_size=64)

W0726 13:37:15.901035 139930500745088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0726 13:37:15.929053 139930500745088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0726 13:37:16.057057 139930500745088 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
135937/135937 [==============================] - 557s 4ms/step - loss: 2.6353
Epoch 2/10
135937/135937 [==============================] - 544s 4ms/step - loss: 2.2184
Epoch 3/10
135937/135937 [==============================] - 545s 4ms/step - loss: 2.0162
Epoch 4/10
135937/135937 [==============================] - 544s 4ms/step - loss: 1.8761
Epoch 5/10
135937/135937 [==============================] - 553s 4ms/step - loss: 1.7738
Epoch 6/10
135937/135937 [==============================] - 543s 4ms/step - loss: 1.6911
Epoch 7/10
135937/135937 [==============================] - 543s 4ms/step - loss: 1.6263
Epoch 8/10
135937/135937 [==============================] - 549s 4ms/step - loss: 1.5645
Epoch 9/10
135937/135937 [==============================] - 549s 4ms/step - loss: 1.5118
Epoch 10/10
135937/135937 [==============================] - 558s 4ms/step - loss: 1.4613


### Saved the model in local drive for every 10 epochs

In [9]:
# Save the trained weights in to .h5 format
model.save("LSTM_model_1.h5")
print("Saved model to disk")


W0726 15:24:42.353788 140090655033216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_1.h5')

### Upload the model from local drive to colab 

In [0]:
#2nd 5 epochs  => 10+5 = 15

from google.colab import files
model_new = files.upload()

### load the model -> compile-> fit ->save -> download to local drive

In [14]:
from keras.models import load_model

model_2 = load_model("LSTM_model_1.h5")
model_2.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_2.fit(X, y, epochs=5, batch_size=64)

model_2.save("LSTM_model_2.h5")
print("Saved model to disk")

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Epoch 1/5
135937/135937 [==============================] - 562s 4ms/step - loss: 2.6750
Epoch 2/5
135937/135937 [==============================] - 581s 4ms/step - loss: 2.2643
Epoch 3/5
135937/135937 [==============================] - 580s 4ms/step - loss: 2.0548
Epoch 4/5
135937/135937 [==============================] - 578s 4ms/step - loss: 1.9073
Epoch 5/5
135937/135937 [==============================] - 587s 4ms/step - loss: 1.8086
Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_2.h5')

In [0]:
#3rd 5 epochs  => 15+5 = 20

from google.colab import files
model_new = files.upload()

In [18]:
from keras.models import load_model

model_3 = load_model("LSTM_model_2.h5")
model_3.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_3.fit(X, y, epochs=5, batch_size=64)

model_3.save("LSTM_model_3.h5")
print("Saved model to disk")

Epoch 1/5
135937/135937 [==============================] - 583s 4ms/step - loss: 1.7260
Epoch 2/5
135937/135937 [==============================] - 576s 4ms/step - loss: 1.6601
Epoch 3/5
135937/135937 [==============================] - 575s 4ms/step - loss: 1.6036
Epoch 4/5
135937/135937 [==============================] - 575s 4ms/step - loss: 1.5537
Epoch 5/5
135937/135937 [==============================] - 569s 4ms/step - loss: 1.5494
Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_3.h5')

In [2]:
#4th 5 epochs  => 20+5 = 25

from google.colab import files
model_new = files.upload()

Saving LSTM_model_3.h5 to LSTM_model_3.h5


In [13]:
from keras.models import load_model

model_4 = load_model("LSTM_model_3.h5")
model_4.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_4.fit(X, y, epochs=5, batch_size=64)

model_4.save("LSTM_model_4.h5")
print("Saved model to disk")

Epoch 1/5
135937/135937 [==============================] - 600s 4ms/step - loss: 1.5003
Epoch 2/5
135937/135937 [==============================] - 592s 4ms/step - loss: 1.4397
Epoch 3/5
135937/135937 [==============================] - 591s 4ms/step - loss: 1.3959
Epoch 4/5
135937/135937 [==============================] - 586s 4ms/step - loss: 1.3576
Epoch 5/5
135937/135937 [==============================] - 583s 4ms/step - loss: 1.3258
Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_4.h5')

In [0]:
#5th 5 epochs  => 25+5 = 30

from google.colab import files
model_new = files.upload()

In [15]:
from keras.models import load_model

model_5 = load_model("LSTM_model_4.h5")
model_5.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_5.fit(X, y, epochs=5, batch_size=64)

model_5.save("LSTM_model_5.h5")
print("Saved model to disk")

Epoch 1/5
135937/135937 [==============================] - 600s 4ms/step - loss: 1.2941
Epoch 2/5
135937/135937 [==============================] - 602s 4ms/step - loss: 1.2624
Epoch 3/5
135937/135937 [==============================] - 588s 4ms/step - loss: 1.2330
Epoch 4/5
135937/135937 [==============================] - 593s 4ms/step - loss: 1.2097
Epoch 5/5
135937/135937 [==============================] - 598s 4ms/step - loss: 1.2507
Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_5.h5')

In [0]:
#6th 5 epochs  => 30+5 = 35

from google.colab import files
model_new = files.upload()

In [17]:
from keras.models import load_model

model_6 = load_model("LSTM_model_5.h5")
model_6.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_6.fit(X, y, epochs=5, batch_size=64)

model_6.save("LSTM_model_6.h5")
print("Saved model to disk")

Epoch 1/5
135937/135937 [==============================] - 605s 4ms/step - loss: 1.2796
Epoch 2/5
135937/135937 [==============================] - 604s 4ms/step - loss: 1.2481
Epoch 3/5
135937/135937 [==============================] - 591s 4ms/step - loss: 1.2281
Epoch 4/5
135937/135937 [==============================] - 587s 4ms/step - loss: 1.2025
Epoch 5/5
135937/135937 [==============================] - 590s 4ms/step - loss: 1.1818
Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_6.h5')

In [0]:
#7th 5 epochs  => 35+5 = 40

from google.colab import files
model_new = files.upload()

In [19]:
from keras.models import load_model

model_7 = load_model("LSTM_model_6.h5")
model_7.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_7.fit(X, y, epochs=5, batch_size=64)

model_7.save("LSTM_model_7.h5")
print("Saved model to disk")

Epoch 1/5
135937/135937 [==============================] - 583s 4ms/step - loss: 1.1643
Epoch 2/5
135937/135937 [==============================] - 583s 4ms/step - loss: 1.1438
Epoch 3/5
135937/135937 [==============================] - 590s 4ms/step - loss: 1.1302
Epoch 4/5
135937/135937 [==============================] - 578s 4ms/step - loss: 1.1220
Epoch 5/5
135937/135937 [==============================] - 567s 4ms/step - loss: 1.1187
Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_7.h5')

In [0]:
#8th 5 epochs  => 40+5 = 45

from google.colab import files
model_new = files.upload()

In [22]:
from keras.models import load_model

model_8 = load_model("LSTM_model_7.h5")
model_8.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_8.fit(X, y, epochs=5, batch_size=64)

model_8.save("LSTM_model_8.h5")
print("Saved model to disk")

Epoch 1/5
135937/135937 [==============================] - 581s 4ms/step - loss: 1.0984
Epoch 2/5
135937/135937 [==============================] - 567s 4ms/step - loss: 1.0867
Epoch 3/5
135937/135937 [==============================] - 561s 4ms/step - loss: 1.0756
Epoch 4/5
135937/135937 [==============================] - 558s 4ms/step - loss: 1.0684
Epoch 5/5
135937/135937 [==============================] - 560s 4ms/step - loss: 1.0559
Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_8.h5')

In [0]:
#9th 5 epochs  => 45+5 = 50

from google.colab import files
model_new = files.upload()

In [25]:
from keras.models import load_model

model_9 = load_model("LSTM_model_8.h5")
model_9.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_9.fit(X, y, epochs=5, batch_size=64)

model_9.save("LSTM_model_9.h5")
print("Saved model to disk")

Epoch 1/5
135937/135937 [==============================] - 571s 4ms/step - loss: 1.0489
Epoch 2/5
135937/135937 [==============================] - 564s 4ms/step - loss: 1.0390
Epoch 3/5
135937/135937 [==============================] - 562s 4ms/step - loss: 1.0325
Epoch 4/5
135937/135937 [==============================] - 565s 4ms/step - loss: 1.0240
Epoch 5/5
135937/135937 [==============================] - 563s 4ms/step - loss: 1.0169
Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_9.h5')

As the train is slow, and in loss difference is less between epochs
lets try to use learning rate in adam for fast learning-- loss increased drastically

Increase batch  size for fast learning
But authour told to keep as minimum as possible.
lets see what happens

Loss reducing - fast training eta of 6mins  previous batchsize =64 eta was 9.5mins ----> observation: good to go with increasing batch size

In [0]:
#10th 5 epochs  => 50+5 = 55

from google.colab import files
model_new = files.upload()

In [28]:
from keras.models import load_model
from keras import optimizers
model_10 = load_model("LSTM_model_9.h5")

#adam1 = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_10.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_10.fit(X, y, epochs=5, batch_size=100)

model_10.save("LSTM_model_10.h5")
print("Saved model to disk")

Epoch 1/5
135937/135937 [==============================] - 389s 3ms/step - loss: 0.9720
Epoch 2/5
135937/135937 [==============================] - 389s 3ms/step - loss: 0.9495
Epoch 3/5
135937/135937 [==============================] - 390s 3ms/step - loss: 0.9336
Epoch 4/5
135937/135937 [==============================] - 389s 3ms/step - loss: 0.9277
Epoch 5/5
135937/135937 [==============================] - 385s 3ms/step - loss: 0.9180
Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_10.h5')

In [0]:
#11th 5 epochs  => 55+5 = 60

from google.colab import files
model_new = files.upload()

In [31]:
from keras.models import load_model
from keras import optimizers
model_11 = load_model("LSTM_model_10.h5")

#adam1 = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_11.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_11.fit(X, y, epochs=5, batch_size=100)

model_11.save("LSTM_model_11.h5")
print("Saved model to disk")

Epoch 1/5
135937/135937 [==============================] - 397s 3ms/step - loss: 0.9152
Epoch 2/5
135937/135937 [==============================] - 391s 3ms/step - loss: 0.9056
Epoch 3/5
135937/135937 [==============================] - 391s 3ms/step - loss: 0.8961
Epoch 4/5
135937/135937 [==============================] - 383s 3ms/step - loss: 0.8947
Epoch 5/5
135937/135937 [==============================] - 379s 3ms/step - loss: 0.8864
Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_11.h5')

In [0]:
#12th 5 epochs  => 60+5 = 65

from google.colab import files
model_new = files.upload()

In [34]:
from keras.models import load_model
from keras import optimizers
model_12 = load_model("LSTM_model_11.h5")

#adam1 = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_12.compile(loss='categorical_crossentropy', optimizer='adam')
# fit the model
model_12.fit(X, y, epochs=5, batch_size=200)

model_12.save("LSTM_model_12.h5")
print("Saved model to disk")

Epoch 1/5
135937/135937 [==============================] - 208s 2ms/step - loss: 0.8335
Epoch 2/5
135937/135937 [==============================] - 201s 1ms/step - loss: 0.8095
Epoch 3/5
135937/135937 [==============================] - 196s 1ms/step - loss: 0.7977
Epoch 4/5
135937/135937 [==============================] - 200s 1ms/step - loss: 0.7856
Epoch 5/5
135937/135937 [==============================] - 198s 1ms/step - loss: 0.7805
Saved model to disk


In [0]:
from google.colab import files
files.download('LSTM_model_12.h5')

#### pattern.append(index) : not working hence using np.append

In [39]:
import sys
import numpy as np
from keras.models import load_model
int_to_char = dict((i, c) for i, c in enumerate(chars))

# pick a random seed
start = numpy.random.randint(0, len(dataX)+1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = (x / float(n_vocab))
    prediction = model_12.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern = np.append(pattern,index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" proved it at all however she went on and how
do you know that youre mad

to begin with said the cat  "
i hive ieard a little betides sr converslog in the walk asg all torrise

whe fuog of thoined the had nookingny done what she wale uuch a thiht seroled turping tuuprised thesh yould be ourerf the fver setfaked hew mnnw the walted a little bnteadid the waid tuceen and where where suoeiers thall ever hetting tumed hereranly have anl have anyays good ou narehned tolebody else the had slease sarty suick and temovel taid to the
whoug and the hot keading ano ano tour

the waited po the fru out heart horesatissed out what she wale tuceeng
inp tuay tuch a nittle whryg antws hor hat byt when whe qtpccsume wioles hot have yonder as ie b gaw what ie soease serpled ovt hetting anl tound het
monw the walted a little bnteadid the waid tound anice

anice suoted intor
the wale tuokshe

alice whought the havter way only twitting how iardent have yonder at sueer soanle anice hestelf and whe grog